  \begin{equation}\label{eq:fput}
    \ddot{x}_{\ell}=f(x_{\ell+1}-x_{\ell})-f(x_{\ell}-x_{\ell-1})\,.
  \end{equation}
    $F$ is an antiderivative of $f$, $F(x)=\frac{1}{2}kx^{2}+\frac{1}{3}\alpha x^{3}$, $x_{11}=k, x_{12}=\alpha$

In [7]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'../../../Data/FPUT/FPUT_lattices_L5_T5_N100'
def generate_random_values():
    min_val = 0
    max_val = 2
    x1 = random.uniform(min_val, max_val)
    x2 = random.uniform(min_val, max_val)
    x3 = random.uniform(min_val, max_val)
    x4 = random.uniform(min_val, max_val)
    x5 = random.uniform(min_val, max_val)
    x6 = random.uniform(min_val, max_val)
    x7 = random.uniform(min_val, max_val)
    x8 = random.uniform(min_val, max_val)
    x9 = random.uniform(min_val, max_val)
    x10 = random.uniform(min_val, max_val)
    x11= random.uniform(min_val, max_val)
    x12 = random.uniform(min_val, max_val)
    return x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = y
        f = np.array([x6, x7, x8,x9, x10, x11 * (x2 - x1) + x12 * (x2 - x1)**2 - x11 * (x1 - x5) - x12 * (x1 - x5)**2, x11 * (x3 - x2) + x12 * (x3 - x2)**2 - x11 * (x2 - x1) - x12* (x2 - x1)**2, x11 * (x4 - x3) + x12 * (x4 - x3)**2 - x11 * (x3 - x2) - x12 * (x3 - x2)**2, x11 * (x5 - x4) + x12 * (x5 - x4)**2 - x11 * (x4 - x3) - x12 * (x4 - x3)**2, x11 * (x1 - x5) + x12 * (x1 - x5)**2 - x11 * (x5 - x4) - x12 * (x5 - x4)**2, 0, 0])
        normalized_f = normalize(f)
        return normalized_f
    num_trajectories = 5
    t = np.linspace(0, 1, 100) # 10 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4, x5, x6):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]}, {initial_condition[4]}, {initial_condition[5]}, {initial_condition[6]}, {initial_condition[7]}, {initial_condition[8]}, {initial_condition[9]}, {initial_condition[10]}, {initial_condition[11]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
    num_variables = 12 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                x1 = data[j, 0]
                x2 = data[j, 1]
                x3 = data[j, 2]
                x4 = data[j, 3]
                x5 = data[j, 4]
                x6 = data[j, 5]
                x7 = data[j, 6]
                x8 = data[j, 7]
                x9 = data[j, 8]
                x10 = data[j, 9]
                x11 = data[j, 10]
                x12 = data[j, 11]
                writer.writerow([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, r+1]) 
    output_directory1 = r'../../../results/FPUT/FPUT_lattices_L5_T5_N100'
def split_data():
    trajectories = {}
    column_names = None
    with open('../../../Data/FPUT/FPUT_lattices_L5_T5_N100/trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = generate_random_values()
    initial_conditions = [generate_random_values() for _ in range(5)]
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4, x5, x6):
(1.694114929749955, 1.8349368928486518, 0.269153052617223, 1.8306276599607003, 1.3218774687513646, 1.1333253465120423, 0.5706484412957418, 1.2249238686448993, 1.1683686623428922, 0.164387182835517, 1.7180561107234236, 1.1837006619102877)
(0.2287619907632572, 1.3582711172844444, 1.0208736826322498, 1.581607612608602, 1.4892721417716595, 0.32197547435204754, 1.604782036357855, 0.9336330462648059, 1.5526354602590378, 0.467158306402796, 1.0613032426495368, 1.3652255746171775)
(1.9198922600181054, 1.3067734648858857, 1.3435561200656392, 0.008311987131933485, 0.24391366214977284, 0.3415188547248498, 0.49458171208140955, 0.992318681608038, 1.1142994477691621, 1.5723469355927935, 1.8899138159067557, 1.77516569052549)
(0.9679766745446698, 0.055755426441289835, 0.6685133506287158, 0.286335387572503, 1.0225360664527905, 1.3759710452502607, 1.352625080200328, 1.3719895441778298, 1.6836551348460667, 1.8275644400346995, 1.5466718854380839, 0.3967675447711019)
(

In [1]:
import pandas as pd
import os
input_directory = r'../../../System/FPUT/FPUT_lattices_L5_T5_N100'
output_directory = r'../../../Data/FPUT/FPUT_lattices_L5_T5_N100/Traj_data'
df14 = pd.read_csv(os.path.join(input_directory, '50.csv')) # 50.csv, saved names of the data
Br = pd.read_csv('../../../System/FPUT/FPUT_lattices_L5_T5_N100/50.csv')
tr = Br.groupby('trajectory').size()
re1 = int(round(tr.mean()))
rows_per_file = re1 
num_files = len(df14) // rows_per_file
data_chunks = [df14.iloc[i * rows_per_file:(i + 1) * rows_per_file].iloc[:, :-1] for i in range(num_files)]
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i, chunk in enumerate(data_chunks):
    chunk.to_csv(os.path.join(output_directory, f'tr_{i + 1}.csv'), index=False)